## Load data about songs


In [1]:
import graphlab

In [2]:
songs = graphlab.SFrame('song_data.gl/')

2016-03-31 07:23:52,314 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1459409031.log


This non-commercial license of GraphLab Create is assigned to gdwangh@189.cn and will expire on February 23, 2017. For commercial licensing options, visit https://dato.com/buy/.


## 1.Counting unique users

Compute the number of unique users for each of these artists: 'Kanye West', 'Foo Fighters', 'Taylor Swift' and 'Lady GaGa'.

In [16]:
for artist_name in ['Kanye West', 'Foo Fighters', 'Taylor Swift' , 'Lady GaGa']:
    print "%s has unique users: %d." % (artist_name, len(songs[songs['artist']==artist_name]['user_id'].unique()))

Kanye West has unique users: 2522.
Foo Fighters has unique users: 2055.
Taylor Swift has unique users: 3246.
Lady GaGa has unique users: 2928.


## 2.Using groupby-aggregate to find the most popular and least popular artist

Compute the number of times any song by every artist was listened to and sort it to find the artist with the most popular and least popular artist in the dataset.

In [23]:
artists_songs_count = songs.groupby(key_columns='artist', operations={'total_count': graphlab.aggregate.SUM('listen_count')}).sort('total_count')

In [24]:
artists_songs_count[0]

{'artist': 'William Tabbert', 'total_count': 14}

In [25]:
artists_songs_count[-1]

{'artist': 'Kings Of Leon', 'total_count': 43218}

## 3.Using groupby-aggregate to find the most recommended songs

let’s use to find the song that is most recommended by the personalized_model model we learned in the course iPython notebook.

### train the model

In [40]:
train_data,test_data = songs.random_split(.8,seed=0)

In [44]:
personalized_model = graphlab.item_similarity_recommender.create(train_data, 
                                                      user_id = 'user_id',
                                                      item_id = 'song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 7000            | 2.02296         |

| 8000            | 2.14599         |

| 9000            | 2.26042         |

+-----------------+-----------------+

Finished training in 2.575s

### make one recommendations for  each of test users

Next, we are going to make recommendations for the users in the test data, but there are over 200,000 users (58,628 unique users) in the test set. Computing recommendations for these many users can be slow in some computers. Thus, we will use only the first 10,000 users only in this question. Using this command to select this subset of users:

In [43]:
subset_test_users = test_data['user_id'].unique()[0:10000]

Let’s compute one recommended song for each of these test users. Use this command to compute these recommendations:

In [47]:
test_recommendations = personalized_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 759.858

recommendations finished on 2000/10000 queries. users per second: 752.44

recommendations finished on 3000/10000 queries. users per second: 757.524

recommendations finished on 4000/10000 queries. users per second: 763.178

recommendations finished on 5000/10000 queries. users per second: 766.72

recommendations finished on 6000/10000 queries. users per second: 769.833

recommendations finished on 7000/10000 queries. users per second: 772.819

recommendations finished on 8000/10000 queries. users per second: 775.817

recommendations finished on 9000/10000 queries. users per second: 773.788

recommendations finished on 10000/10000 queries. users per second: 774.512

### find the most recommended song

In [52]:
test_recommendations.groupby(key_columns = 'song', 
                             operations={'count': graphlab.aggregate.COUNT()}).sort('count', ascending=False)

song,count
Undo - Björk,447
Secrets - OneRepublic,373
Revelry - Kings Of Leon,228
You're The One - DwightYoakam ...,179
Fireflies - CharttraxxKaraoke ...,124
Hey_ Soul Sister - Train,117
Horn Concerto No. 4 in Eflat K495: II. Romance ...,95
OMG - Usher featuringwill.i.am ...,68
Sehr kosmisch - Harmonia,66
Dog Days Are Over (RadioEdit) - Florence + The ...,52
